In [97]:
import pandas as pd
import numpy as np
import scratch

data = pd.read_excel("Folds5x2_pp.xlsx")
data.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
AT    9568 non-null float64
V     9568 non-null float64
AP    9568 non-null float64
RH    9568 non-null float64
PE    9568 non-null float64
dtypes: float64(5)
memory usage: 373.8 KB


In [98]:
train = data[0:7176]
test = data[7176:9568]
train.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [114]:
y_test = test['PE']
y_test1 = np.array(y_test)
y_test = y_test1
y_test

array([474.58, 435.87, 447.2 , ..., 429.57, 435.74, 453.28])

In [75]:
# checking correlation using Pearson's correlation

from scipy.stats import pearsonr

for i in range(train.shape[1]-1):
    feature_name = train.columns[i]
    X = train[feature_name].values
    Y = train['PE']
    print(feature_name, ":", pearsonr(X,Y))

AT : (-0.9487138524721694, 0.0)
V : (-0.8687149447262569, 0.0)
AP : (0.5227594077060119, 0.0)
RH : (0.3825704146913939, 7.469842250567013e-249)


In [72]:
test['PE'] = np.nan
test.head()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,AT,V,AP,RH,PE
7176,10.52,41.78,1013.54,71.52,NaN
7177,26.90,60.93,1007.10,67.32,NaN
7178,24.86,44.05,1005.69,66.65,NaN
7179,13.39,49.83,1007.14,90.88,NaN
7180,27.98,71.98,1005.58,81.00,NaN


In [74]:
data = pd.concat([train, test], axis=0, ignore_index=True)
data.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [76]:
y = data.values[:,4]
del data['PE']
y

array([463.26, 444.37, 488.56, ...,    nan,    nan,    nan])

In [78]:
np.any(np.isnan(data.values[:,0:data.shape[1]-1]))

False

In [79]:
np.all(np.isfinite(data.values[:,0:data.shape[1]-1]))

True

merging train and test data for scaling, normalization etc.

In [80]:
from sklearn import preprocessing

x_scaled = preprocessing.scale(data.values[:,0:data.shape[1]])
data1 = pd.DataFrame(x_scaled, columns=data.columns[0:data.shape[1]])
data1.head()

,AT,V,AP,RH
0,-0.629519,-0.987297,1.820488,-0.009519
1,0.741909,0.681045,1.141863,-0.974621
2,-1.951297,-1.173018,-0.185078,1.289840
3,0.162205,0.237203,-0.508393,0.228160
4,-1.185069,-1.322539,-0.678470,1.596699


In [81]:
data1['AT_AT'] = data1['AT'] ** 2
data1['V_V'] = data1['V'] ** 2
data1['AP_AP'] = data1['AP'] ** 2
data1['RH_RH'] = data1['RH'] ** 2


data1['AT_V'] = data1['AT'] * data1['V']
data1['V_AP'] = data1['V'] * data1['AP']
data1['AP_RH'] = data1['AP'] * data1['RH']
data1['RH_AT'] = data1['RH'] * data1['AT']

data1.describe()

,AT,V,AP,RH,AT_AT,V_V,AP_AP,RH_RH,AT_V,V_AP,AP_RH,RH_AT
count,9.568000e+03,9.568000e+03,9.568000e+03,9.568000e+03,9.568000e+03,9.568000e+03,9.568000e+03,9.568000e+03,9568.000000,9568.000000,9568.000000,9568.000000
mean,-1.719813e-16,-2.600229e-16,5.504864e-15,-3.490855e-16,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.844107,-0.413502,0.099574,-0.542535
std,1.000052e+00,1.000052e+00,1.000052e+00,1.000052e+00,9.810538e-01,7.455529e-01,1.446990e+00,1.247093e+00,0.772026,0.963885,0.951216,0.991560
min,-2.394126e+00,-2.277901e+00,-3.430019e+00,-3.270589e+00,2.729561e-08,2.085986e-07,2.409601e-08,3.781531e-07,-2.090820,-4.757947,-5.260527,-6.765312
25%,-8.240958e-01,-9.888705e-01,-7.003615e-01,-6.836860e-01,2.104670e-01,4.178577e-01,1.055237e-01,1.297557e-01,0.199246,-1.004796,-0.320979,-1.046576
50%,9.309729e-02,-1.751604e-01,-5.373067e-02,1.141150e-01,6.719732e-01,9.654534e-01,4.722445e-01,5.585404e-01,0.740092,-0.251820,0.019083,-0.263604
75%,8.143721e-01,9.627745e-01,6.737290e-01,7.891378e-01,1.554501e+00,1.379368e+00,1.295464e+00,1.394600e+00,1.385682,0.175795,0.468218,0.077848
max,2.342804e+00,2.144779e+00,3.374760e+00,1.839173e+00,5.731842e+00,5.188831e+00,1.176503e+01,1.069675e+01,3.906942,3.529507,5.072179,2.787771


In [82]:
data2 = pd.DataFrame(y, columns=train.columns[train.shape[1]-1:train.shape[1]])
data2 = data1.join(data2)
data2.head()

,AT,V,AP,RH,AT_AT,V_V,AP_AP,RH_RH,AT_V,V_AP,AP_RH,RH_AT,PE
0,-0.629519,-0.987297,1.820488,-0.009519,0.396295,0.974755,3.314178,0.000091,0.621522,-1.797362,-0.017330,0.005993,463.26
1,0.741909,0.681045,1.141863,-0.974621,0.550429,0.463822,1.303851,0.949885,0.505274,0.777660,-1.112883,-0.723080,444.37
2,-1.951297,-1.173018,-0.185078,1.289840,3.807561,1.375970,0.034254,1.663686,2.288906,0.217099,-0.238720,-2.516861,488.56
3,0.162205,0.237203,-0.508393,0.228160,0.026311,0.056265,0.258463,0.052057,0.038476,-0.120592,-0.115995,0.037009,446.48
4,-1.185069,-1.322539,-0.678470,1.596699,1.404388,1.749109,0.460322,2.549449,1.567299,0.897303,-1.083313,-1.892198,473.90


In [85]:
data_train = data2[0:7176]
data_test = data2[7176:9568]

In [86]:
del data_test['PE']
data_test.head()

,AT,V,AP,RH,AT_AT,V_V,AP_AP,RH_RH,AT_V,V_AP,AP_RH,RH_AT
7176,-1.225326,-0.985723,0.047305,-0.122537,1.501424,0.971649,0.002238,0.015015,1.207832,-0.046630,-0.005797,0.150148
7177,0.972717,0.521294,-1.037148,-0.410218,0.946178,0.271747,1.075677,0.168279,0.507071,-0.540659,0.425457,-0.399026
7178,0.698968,-0.807084,-1.274583,-0.456110,0.488556,0.651385,1.624562,0.208036,-0.564126,1.028696,0.581350,-0.318806
7179,-0.840199,-0.352225,-1.030413,1.203535,0.705934,0.124062,1.061750,1.448497,0.295939,0.362937,-1.240138,-1.011209
7180,1.117643,1.390877,-1.293106,0.526800,1.249126,1.934540,1.672124,0.277518,1.554504,-1.798552,-0.681209,0.588774


In [87]:
data_test = data_test.reset_index(drop=True)
data_test.head()

,AT,V,AP,RH,AT_AT,V_V,AP_AP,RH_RH,AT_V,V_AP,AP_RH,RH_AT
0,-1.225326,-0.985723,0.047305,-0.122537,1.501424,0.971649,0.002238,0.015015,1.207832,-0.046630,-0.005797,0.150148
1,0.972717,0.521294,-1.037148,-0.410218,0.946178,0.271747,1.075677,0.168279,0.507071,-0.540659,0.425457,-0.399026
2,0.698968,-0.807084,-1.274583,-0.456110,0.488556,0.651385,1.624562,0.208036,-0.564126,1.028696,0.581350,-0.318806
3,-0.840199,-0.352225,-1.030413,1.203535,0.705934,0.124062,1.061750,1.448497,0.295939,0.362937,-1.240138,-1.011209
4,1.117643,1.390877,-1.293106,0.526800,1.249126,1.934540,1.672124,0.277518,1.554504,-1.798552,-0.681209,0.588774


In [88]:
data_train_1 = data_train.values
data_train_1

array([[-6.29519384e-01, -9.87296587e-01,  1.82048840e+00, ...,
        -1.73298709e-02,  5.99261697e-03,  4.63260000e+02],
       [ 7.41909107e-01,  6.81045124e-01,  1.14186280e+00, ...,
        -1.11288291e+00, -7.23079836e-01,  4.44370000e+02],
       [-1.95129733e+00, -1.17301765e+00, -1.85077563e-01, ...,
        -2.38720389e-01, -2.51686077e+00,  4.88560000e+02],
       ...,
       [ 1.69197894e+00,  1.52465942e+00,  2.51061466e-01, ...,
        -2.13391892e-01, -1.43811233e+00,  4.26480000e+02],
       [ 1.23573072e+00,  1.50262472e+00, -3.18108387e-01, ...,
         3.17007435e-01, -1.23145393e+00,  4.39330000e+02],
       [-1.39172230e+00, -1.10848745e+00, -6.78470364e-01, ...,
        -7.02706350e-01, -1.44143673e+00,  4.80730000e+02]])

In [89]:
data_test_1 = data_test.values
data_test_1

array([[-1.22532589, -0.98572268,  0.04730539, ..., -0.04663   ,
        -0.00579666,  0.15014761],
       [ 0.97271703,  0.52129354, -1.03714841, ..., -0.54065876,
         0.42545679, -0.39902589],
       [ 0.6989681 , -0.8070842 , -1.27458317, ...,  1.02869594,
         0.58134989, -0.31880621],
       ...,
       [ 1.56583973,  1.57581141, -0.05709854, ..., -0.08997654,
         0.14403782, -3.95001567],
       [ 0.64797565,  1.19177803,  0.1011913 , ...,  0.12059757,
        -0.07568102, -0.48462132],
       [ 0.26150656,  0.64641916,  0.66867722, ...,  0.43224577,
        -0.24911258, -0.09742305]])

In [90]:
def step_grad(points, learn_rate, M):
    N = points.shape[0]
    num_col = points.shape[1]
    
    new_M = np.zeros(num_col)
    
    for i in range(N):
        x = points[i, 0:num_col-1]
        x = np.append(x, 1)
        y = points[i, num_col-1]
        for j in range(num_col):
            new_M[j] += (-2/N) * (y - (M * x).sum()) * x[j]
        M = M - (learn_rate * new_M)
    
    return M

In [91]:
def grad_desc(points, learn_rate, num_iter):
    num_col = points.shape[1]
    M = np.zeros(num_col)
    
    for i in range(num_iter):
        M = step_grad(points, learn_rate, M)
        if i % 100== 0:
            print(i, "Cost= ", cost(points, M))
    print(i, "Cost= ", cost(points, M))
    return M

In [92]:
def cost(points, M):
    total_cost = 0
    
    N = points.shape[0]
    num_col = points.shape[1]
    
    for i in range(N):
        x = points[i,0:num_col-1]
        x = np.append(x, 1)
        y = points[i,num_col-1]
        total_cost += (y - (M * x).sum()) ** 2
        
    total_cost = (1/N) * total_cost
    return total_cost

In [93]:
def run():
    data = data_train_1[:,0:data_train_1.shape[1]]
    # tweak learn_rate & num_iter to get better results; 0.001, 1000
    learn_rate = 0.0001
    num_iter = 1000
    
    m = grad_desc(data, learn_rate, num_iter)
    #print(m)
    return m[0:data_train_1.shape[1]-1], m[data_train_1.shape[1]-1]

In [94]:
m, c = run()
print(m, c)

0 Cost=  150916.78726182066
100 Cost=  19.22801463585786
200 Cost=  17.760785355220417
300 Cost=  17.743503437259577
400 Cost=  17.742755241970794
500 Cost=  17.742698910240094
600 Cost=  17.742694375174196
700 Cost=  17.742694007998605
800 Cost=  17.74269397825706
900 Cost=  17.74269397584759
999 Cost=  17.74269397565332
[-13.61642886  -3.6311436    0.79233231  -1.91396256   0.92490933
  -0.12851954  -0.25278515  -0.3947235    1.00681491   0.30883716
  -0.40036628  -0.6160933 ] 453.1648310371076


In [36]:
data_test_1.shape

(2392, 4)

In [95]:
def predict(x, m, c):
    N = x.shape[0]
    y = np.zeros(N)
    
    for i in range(N):
        X = x[i,:]
        y[i] = (m * X).sum() + c
        
    return y

In [116]:
y_pred = predict(data_test_1, m, c)
y_pred

array([476.06949677, 438.91129813, 446.02986713, ..., 435.17296765,
       442.65187733, 448.80483986])

In [115]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import r2_score

#模型拟合测试集

from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",mean_absolute_error(y_test, y_pred))
# 用scikit-learn计算R2
print("R2:",r2_score(y_test, y_pred, multioutput='raw_values'))

MSE: 19.283391319352173
RMSE: 4.391285838948789
MAE: 3.397251451021495
R2: [0.9335685]
